In [2]:
pip install astroquery

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 67.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.6/997.6 kB 39.6 MB/s eta 0:00:00


In [7]:
object_name = 'Mkn 501'
swift_url = "https://www.swift.ac.uk/user_objects/tprods/USERPROD_211576/spec/"
output_file = "Mkn501_xrt.dat"

In [8]:
from astroquery.heasarc import Heasarc
from astropy.coordinates import SkyCoord
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re

In [9]:
pos = SkyCoord.from_name(object_name)
heasarc_table = Heasarc.query_region(pos, catalog='swiftmastr')

In [10]:
response = requests.get(swift_url)
soup = BeautifulSoup(response.text, 'html.parser')

tables = soup.find_all('table', class_='chTable')

data_list = []

for table in tables:
    summary = table.get('summary', '')
    obs_id = summary.split("_")[-1].split()[0] if "Obs_" in summary else "Unknown"

    matched_row = heasarc_table[heasarc_table['obsid'] == obs_id]
    time = matched_row['start_time'][0]

    photon_index, photon_upper, photon_lower = None, None, None
    flux_obs, flux_upper, flux_lower = None, None, None
    chisq, dof, redchisq = None, None, None

    for row in table.find_all('tr'):
        th = row.find('th')
        td = row.find('td')
        if not th or not td:
            continue

        key = th.get_text(strip=True)
        value = td.get_text(strip=True)

        if "Photon index" in key:
            parts = value.split(" ")
            photon_index = float(parts[0])
            if "(" in value:
                errors = value[value.find("(") + 1:value.find(")")].split(",")
                photon_upper = float(errors[0].strip())
                photon_lower = float(errors[1].strip())

        elif "Flux (0.3-10 keV)(Observed)" in key:
            match = re.search(r"([\d.]+)\s*\(\+?([\d.]+),\s*-?([\d.]+)\)\s*×\s*10\^?(-?\d+)", value)
            if match:
                base_flux = float(match.group(1))
                upper_err = float(match.group(2))
                lower_err = float(match.group(3))
                exponent = int(match.group(4))

                flux_obs = base_flux * (10 ** exponent)
                flux_upper = upper_err * (10 ** exponent)
                flux_lower = lower_err * (10 ** exponent)

        elif "W-stat" in key:
            chisq_dof = value.split(" ")
            chisq = float(chisq_dof[0])
            dof = int(chisq_dof[1].strip("()"))
            redchisq = chisq / dof

    data_list.append([
        obs_id, time, photon_index, photon_upper, photon_lower,
        flux_obs, flux_upper, flux_lower, chisq, dof, round(redchisq, 3) if redchisq else None
    ])

with open(output_file, "w") as f:
    f.write("# Obs_ID  Time  Photon_Index  Photon_Upper_Error  Photon_Lower_Error  Flux_Observed  Flux_Upper_Error  Flux_Lower_Error  Chisq  Dof  Redchisq\n")

    for row in data_list:
        f.write("  ".join(str(x) if x is not None else "NA" for x in row) + "\n")

print(f"Data successfully saved to {output_file}")


Data successfully saved to Mkn501_xrt.dat
